In [ ]:
#Импорт библиотек
import os
import mat_properties as prop
import numpy as n
import pandas as pd
import cotel
import nasos
import time
from scipy.optimize import root
from time import sleep
import sys
from PyQt5.QtWidgets import (QMainWindow,QWidget, QHBoxLayout, QLabel, QApplication, QPushButton,QVBoxLayout, QTableWidget)
from PyQt5.QtCore import Qt, QObject, QThread, pyqtSignal
from PyQt5.QtGui import QPixmap

#Основные константы
calctolerance=10**-3
calcmethod = "hybr"
KPDnasos = 0.8074
KPD=0.99

#таблица номинального режима
gas_streams0   = pd.read_excel('streams0.xlsx', sheet_name='gas',  index_col=0)
water_streams0 = pd.read_excel('streams0.xlsx', sheet_name='water',index_col=0)
#таблица задаваемых значений (заменится на интерфейс)
gas_streamsZAD    = pd.read_excel('streams.xlsx',  sheet_name='gas',  index_col=0)
water_streamsZAD  = pd.read_excel('streams.xlsx',  sheet_name='water',index_col=0)
#рабочая таблица (=номинал в 1 итерации)
gas_streams = gas_streams0.copy()
water_streams = water_streams0.copy()
#рабочая таблица показателей блоков
heaters  = pd.read_excel('blocks.xlsx',  sheet_name='heaters',index_col=0)
electric  = pd.read_excel('blocks.xlsx',  sheet_name='electric',index_col=0)


#чтение с вводного экселя входных данных
Tgas = gas_streamsZAD.loc['GTU-PEVD','T']
Pgas = gas_streamsZAD.loc['GTU-PEVD','P']
Ggas = gas_streamsZAD.loc['GTU-PEVD','G']
Propgas = gas_streamsZAD.loc['GTU-PEVD','N2':'Ar']
Pvd = water_streamsZAD.loc['IVD-PEVD','P']
Pnd = water_streamsZAD.loc['IND-PPND','P']
Twat0 = water_streamsZAD.loc['X-GPK','T']

#запись состава газов в раб табл
gas_streams['N2'] = Propgas[0]
gas_streams['O2'] = Propgas[1]
gas_streams['CO2'] = Propgas[2]
gas_streams['H2O'] = Propgas[3]
gas_streams['Ar'] = Propgas[4]

#Состав газов в номинальном режиме и в частичной нагрузке
gasmix = "Nitrogen*Oxygen*CO2*Water*Argon"
fractiongas0 = (gas_streams0.at['GTU-PEVD','N2'],gas_streams0.at['GTU-PEVD','O2'],gas_streams0.at['GTU-PEVD','CO2'],gas_streams0.at['GTU-PEVD','H2O'],gas_streams0.at['GTU-PEVD','Ar'])
fractiongas  = (gas_streams.at['GTU-PEVD','N2'],gas_streams.at['GTU-PEVD','O2'],gas_streams.at['GTU-PEVD','CO2'],gas_streams.at['GTU-PEVD','H2O'],gas_streams.at['GTU-PEVD','Ar'])

#Считывание рефпропа
RP = prop.init_REFPROP(r'C:\Program Files (x86)\REFPROP')
os.environ['RPPREFIX'] = r'C:/Program Files (x86)/REFPROP'
gas1 = prop.Materials_prop(gasmix,fractiongas, prop.REFPROP_h_s,prop.REFPROP_p_t,prop.REFPROP_p_h,prop.REFPROP_p_s,prop.REFPROP_p_q,prop.REFPROP_t_q,RP=RP)
gas0 = prop.Materials_prop(gasmix,fractiongas0, prop.REFPROP_h_s,prop.REFPROP_p_t,prop.REFPROP_p_h,prop.REFPROP_p_s,prop.REFPROP_p_q,prop.REFPROP_t_q,RP=RP)
water=prop.Materials_prop('water',[1.0,0,0,0,0], prop.REFPROP_h_s,prop.REFPROP_p_t,prop.REFPROP_p_h,prop.REFPROP_p_s,prop.REFPROP_p_q,prop.REFPROP_t_q,RP=RP)

#Задание в раб табл параметров газа (1 строка)
gas_streams.loc['GTU-PEVD','T'] = Tgas
gas_streams['G'] = Ggas
gas_streams['P'] = Pgas
gas_streams.loc['GTU-PEVD','H'] = gas1.p_t(Pgas,Tgas)['h']

#Задание энтальпий газа в номинальном режиме
Temperatures= gas_streams0.loc['GTU-PEVD':'GPK-out','T']
Enthalpies=list(map( lambda x: gas1.p_t(Pgas,x)['h'],Temperatures))
gas_streams0.loc['GTU-PEVD':'GPK-out','H'] = Enthalpies

#Задание в раб табл параметров по воде
water_streams.loc['IVD-PEVD':'PEN-EVD','P'] = Pvd
water_streams.loc['IND-PPND':'X-GPK','P'] = Pnd
water_streams.loc['BND-PEN','P'] = Pnd
water_streams.loc['X-GPK','T'] = Twat0

#Расчёт параметров на насыщении в испарительных поверхностях (в 1 итерации приближение: недогрев до н.п. на входе 3°С)
water_streams.loc['IVD-PEVD','T'] = water.p_q(Pvd,1)['T']
water_streams.loc['IVD-PEVD','H'] = water.p_q(Pvd,1)['h']
water_streams.loc['IND-PPND','T'] = water.p_q(Pnd,1)['T']
water_streams.loc['IND-PPND','H'] = water.p_q(Pnd,1)['h']
water_streams.loc['EVD-IVD','T'] = water.p_q(Pvd,1)['T']-3
water_streams.loc['GPK-IND','T'] = water.p_q(Pnd,1)['T']-3
water_streams.loc['EVD-IVD','H'] = water.p_t(Pvd,water_streams.loc['EVD-IVD','T'])['h']
water_streams.loc['GPK-IND','H'] = water.p_t(Pnd,water_streams.loc['GPK-IND','T'])['h']

Whole_cotel=cotel.cotel_all(KPD,KPDnasos, calctolerance,gas0,gas1,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams)

class Worker(QObject):
    finished = pyqtSignal()
    progress = pyqtSignal(float)
    
    def __init__(self,Whole_cotel):
        
        QObject.__init__(self)
        self.Whole_cotel=Whole_cotel
        
    def run(self):
        Whole_cotel.calc( maxiterations=50)
    
class Window(QMainWindow):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.clicksCount = 0
        self.setupUi()

    
    def setupUi(self):
        # дефолтные настройки
        self.setWindowTitle("Поверочный расчет котла")
        self.resize(1000, 500)
        self.centralWidget = QWidget()
        self.setCentralWidget(self.centralWidget)
        # создание кнопок и текста
        self.clicksLabel = QLabel("проверка работоспособности", self)
        self.clicksLabel.setAlignment(Qt.AlignHCenter | Qt.AlignVCenter)
        self.stepLabel = QLabel("Невязки,%: не расчитано")
        self.stepLabel.setAlignment(Qt.AlignHCenter | Qt.AlignVCenter)
        self.countBtn = QPushButton("Click me!", self)
        self.countBtn.clicked.connect(self.countClicks)
        self.longRunningBtn = QPushButton("Long-Running Task!", self)
        self.longRunningBtn.clicked.connect(self.runLongTask)
        #картинка
        self.pixmap = QPixmap("cotl.png")
        self.lbl = QLabel(self)
        self.lbl.setPixmap(self.pixmap)
        
#                 # Отображение таблицы, не доделано 
#         table = QTableWidget()
#         table.setColumnCount(len(headers))
#         table.setHorizontalHeaderLabels(headers)

#         for i, row in df.iterrows():
#             # Добавление строки
#             table.setRowCount(table.rowCount() + 1)

#             for j in range(table.columnCount()):
#                 table.setItem(i, j, QTableWidgetItem(str(row[j])))
#         table.show()
#         water_streams
         
        # Set the layout
        layout = QVBoxLayout()
        layout.addWidget(self.clicksLabel)
        layout.addWidget(self.countBtn)
        layout.addWidget(self.lbl)
        layout.addStretch()
        layout.addWidget(self.stepLabel)
        layout.addWidget(self.longRunningBtn)
        self.centralWidget.setLayout(layout)
        self.show()        

    def countClicks(self):
        self.clicksCount += 1
        self.clicksLabel.setText(f"Counting: {self.clicksCount} clicks")

    def reportProgress(self, n):
        self.stepLabel.setText(f"Невязки,%: {n*100}")

    def runLongTask(self):
        # Step 2: Создаем поточный обьект
        self.thread = QThread()
        # Step 3: Create a рабочий oбьект
        self.worker = Worker(Whole_cotel)
        # Step 4: перемещаем функцию в поток
        self.worker.moveToThread(self.thread)
        # Step 5: Connect signals and slots передаем переменные в поток
        self.thread.started.connect(self.worker.run)
        self.worker.finished.connect(self.thread.quit)
        self.worker.finished.connect(self.worker.deleteLater)
        self.thread.finished.connect(self.thread.deleteLater)
        self.worker.progress.connect(self.reportProgress)
        # Step 6: начинаем обработку
        self.thread.start()

        # сброс потоков после расчета
        self.longRunningBtn.setEnabled(False)
        self.thread.finished.connect(
            lambda: self.longRunningBtn.setEnabled(True)
        )
        self.thread.finished.connect(
            lambda: self.stepLabel.setText("Long-Running Step: 0")
        )

app = QApplication(sys.argv)
win = Window()
win.show()
sys.exit(app.exec())



Prop:--- 0.0 сек. ---
Iter:--- 0.1 сек. ---
Else:--- 0.0 сек. ---
PEVD:--- 0.1 сек. ---
IVD:--- 0.2 сек. ---
Prop:--- 0.0 сек. ---
Iter:--- 0.2 сек. ---
Else:--- 0.0 сек. ---
EVD:--- 0.5 сек. ---
dQ/Q ПЕВД+ИВД+ЭВД -0.32731888771103046
Prop:--- 0.0 сек. ---
Iter:--- 0.1 сек. ---
Else:--- 0.0 сек. ---
PEVD:--- 0.1 сек. ---
IVD:--- 0.2 сек. ---
Prop:--- 0.0 сек. ---
Iter:--- 0.2 сек. ---
Else:--- 0.0 сек. ---
EVD:--- 0.5 сек. ---
dQ/Q ПЕВД+ИВД+ЭВД -0.5400697353503915
Prop:--- 0.0 сек. ---
Iter:--- 0.1 сек. ---
Else:--- 0.0 сек. ---
PEVD:--- 0.1 сек. ---
IVD:--- 0.2 сек. ---
Prop:--- 0.0 сек. ---
Iter:--- 0.2 сек. ---
Else:--- 0.0 сек. ---
EVD:--- 0.5 сек. ---
dQ/Q ПЕВД+ИВД+ЭВД -0.32564984924157375
Prop:--- 0.0 сек. ---
Iter:--- 0.1 сек. ---
Else:--- 0.0 сек. ---
PEVD:--- 0.1 сек. ---
IVD:--- 0.2 сек. ---
Prop:--- 0.0 сек. ---
Iter:--- 0.2 сек. ---
Else:--- 0.0 сек. ---
EVD:--- 0.5 сек. ---
dQ/Q ПЕВД+ИВД+ЭВД -0.2797211318395744
Prop:--- 0.0 сек. ---
Iter:--- 0.1 сек. ---
Else:--- 0.0 сек. 

In [ ]:
#Вывод газовых потоков
gas_streams

In [ ]:
#Вывод водяных потоков
water_streams

In [ ]:
#Невязка по всем параметрам
Qgas = KPD*gas_streams.at['GTU-PEVD','G']*(gas_streams.at['GTU-PEVD','H']-gas_streams.at['PEVD-IVD','H'])
Qwat = water_streams.at['PEVD-DROSVD','G']*(water_streams.at['PEVD-DROSVD','H']-water_streams.at['IVD-PEVD','H'])
print('dQ/Q ПЕВД+ИВД',((Qgas-Qwat)/Qgas*100))

Qgas1 = KPD*gas_streams.at['GTU-PEVD','G']*(gas_streams.at['GTU-PEVD','H']-gas_streams.at['IVD-EVD','H'])
Qwat1 = water_streams.at['PEVD-DROSVD','G']*(water_streams.at['PEVD-DROSVD','H']-water_streams.at['EVD-IVD','H'])
print('dQ/Q ПЕВД+ИВД+ЭВД',((Qgas1-Qwat1)/Qgas1*100))

Qgas = KPD*gas_streams.at['EVD-PPND','G']*(gas_streams.at['EVD-PPND','H']-gas_streams.at['IND-GPK','H'])
Qwat = water_streams.at['IND-PPND','G']*(water_streams.at['IND-PPND','H']-water_streams.at['GPK-IND','H'])+water_streams.at['PPND-DROSND','G']*(water_streams.at['PPND-DROSND','H']-water_streams.at['IND-PPND','H'])+water_streams.at['BND-PEN','G']*(water_streams.at['BND-PEN','H']-water_streams.at['GPK-IND','H'])
print('dQ/Q ППНД+ИНД',((Qgas-Qwat)/Qgas*100))

Qgas1 = KPD*gas_streams.at['EVD-PPND','G']*(gas_streams.at['EVD-PPND','H']-gas_streams.at['GPK-out','H'])
Qwat1 = water_streams.at['GPK-IND','G']*(water_streams.at['GPK-IND','H']-water_streams.at['X-GPK','H'])+water_streams.at['IND-PPND','G']*(water_streams.at['IND-PPND','H']-water_streams.at['GPK-IND','H'])+water_streams.at['PPND-DROSND','G']*(water_streams.at['PPND-DROSND','H']-water_streams.at['IND-PPND','H'])+water_streams.at['BND-PEN','G']*(water_streams.at['BND-PEN','H']-water_streams.at['GPK-IND','H'])
print('dQ/Q ППНД+ИНД+ГПК',(Qgas1-Qwat1)/Qgas1*100)

Qgasall = KPD*gas_streams.at['GTU-PEVD','G']*(gas_streams.at['GTU-PEVD','H']-gas_streams.at['GPK-out','H'])
Qwatall = water_streams.at['PPND-DROSND','G']*(water_streams.at['PPND-DROSND','H']-water_streams.at['X-GPK','H'])+water_streams.at['PEVD-DROSVD','G']*(water_streams.at['PEVD-DROSVD','H']-water_streams.at['X-GPK','H'])-water_streams.at['BND-PEN','G']*(water_streams.at['PEN-EVD','H']-water_streams.at['BND-PEN','H'])
print('dQ/Qsumm',(Qgasall-Qwatall)/Qgasall*100)

In [ ]:
gas_streams0

In [ ]:
electric

In [ ]:
heaters